In [3]:
# Import libraries
import pystan # install with pip install pystan
import pandas as pd
import numpy as np
import seaborn as sns
from scipy.stats import beta
import matplotlib.pyplot as plt
import pickle
import hashlib
import time

def get_stan_model(code):
    code_hash = hashlib.sha1(code.encode('utf-8')).hexdigest()
    cache_path =  './cache/' + code_hash + '.pkl'
    try:
        with open(cache_path, 'rb') as file:
            model = pickle.load(file)
    except Exception:
        model = pystan.StanModel(model_code=code) ## c로 변환
        with open(cache_path, 'wb') as file:
            pickle.dump(model, file)
    return model

회귀 시 주의사항을 알아본다.

## 7.1 상호작용

독립변수 간의 교호작용들 등 곱한 항 고려 - 아르바이트 여부가 score가 출석수 y에 미치는 영향의 정도에 영향을 줄때, 위 식을 아래 식으로 변환

$$\mu [n] =  b_1 + b_2 * Score[n] + b_3 * A[n]  $$
$$\mu [n] =  b_1 + b_2 * A[n] + (b_3 +  b_4 * A[n]) * Score[n] $$

0,1 2개의 값을 가지는 A와 달리 값이 연속적이거나, 3개 이상의 이산값을 가지는 경우, 예를 들어 동아리 가입여부, 3종류 동아리) 다음과 같이 모형화 가능하다. 만약 세번째 동아리에 가입한 학생 수가 적다면 b6의 추정이 어려워질수 있는데, 8장의 계층모형을 이용해 b_4,b_5로부터 정보 빌려올 수 있다.

$$\mu [n] =  b_1 + b_2 * A[n] + (b_3 +  b_4 * C1[n]  +  b_5 * C2[n] +  b_6 * C3[n]) * Score[n] $$

## 7.2 로그적용여부

큰 값을 가지는 데이터의 오차에 영향을 많이 받아 노이즈 분포가 뒤틀리는 경우 방지하고 싶을 때나

복리, 암세포 증식처럼 배수적으로 증가할 때 주로 로그 취한다.

In [4]:
#data
df = pd.read_csv('data/data-rental.txt')

#model
model_7_1 ='''
data {
    int N;
    vector <lower = 0> [N] Area;
    vector <lower = 0> [N] Y;
    int N_new;
    vector <lower = 0> [N_new] Area_new;
    
}
parameters {
    real b[2];
    real <lower = 0> sigma;
}

transformed parameters {
    vector <lower = 0> [N] mu;
    mu = b[1] + b[2] * Area;
}

model {
    Y ~ normal(mu, sigma);
}
        
generated quantities {
    vector [N] Y_pred;
    vector [N_new] Y_new;
    for (i in 1: N)
        Y_pred[i] = normal_rng(mu[i], sigma);
    for (i in 1: N_new)
        Y_new[i] = normal_rng(b[1] + b[2] * Area_new[i], sigma);
}
'''

#python interface

data = {
    'N': df.shape[0],
    'Area' : df.iloc[:,1],
    'Y' : df.iloc[:,0],
    'N_new' :50,
    'Area_new': np.linspace(10, 120, 50)
}

#result
model_7_1 = get_stan_model(model_7_1)
params = model_7_1.sampling(data = data)

In [5]:
#data
df = pd.read_csv('data/data-rental.txt')

#model
model_7_2 ='''
data {
    int N;
    vector <lower = 0> [N] Area;
    vector <lower = 0> [N] Y;
    int N_new;
    vector <lower = 0> [N_new] Area_new;
    
}

parameters {
    real b[2];
    real <lower = 0> sigma;
}

transformed parameters {
    vector <lower = 0> [N] mu;
    mu = b[1] + b[2] * Area;
}

model {
    Y ~ normal(mu, sigma);
}
        
generated quantities {
    vector [N] Y_pred;
    vector [N_new] Y_new;
    for (i in 1: N)
        Y_pred[i] = normal_rng(mu[i], sigma);
    for (i in 1: N_new)
        Y_new[i] = normal_rng(b[1] + b[2] * Area_new[i], sigma);
}
'''

#python interface

data = {
    'N': df.shape[0],
    'Area' : np.log(df.iloc[:,1]),
    'Y' : np.log(df.iloc[:,0]),
    'N_new' :50,
    'Area_new': np.log(np.linspace(10, 120, 50))
}

#result
model_7_2 = get_stan_model(model_7_2)
params = model_7_2.sampling(data = data)


#결과값 Y_new에 10^Y_new을 해야 앞선 Y_new와 비교가 가능하다

## 7.3 비선형 관계

이해가 쉬운 이차식을 이용해, 소비전력과 온도의 관계를 다음과 같이 비선형으로 표현가능하다. 최적이라 느끼는 온도가 x0일때, a,b,x0,$\sigma_Y$ 추정

$$y = a + b * (X[n] - x0)^2$$

In [8]:
#data
df = pd.read_csv('data/data-aircon.txt')

#model
model_7_3 ='''
data {
  int N;
  real X[N];
  real Y[N];
}

parameters {
  real a;
  real<lower=0> b;
  real<lower=0, upper=30> x0;
  real<lower=0> s_Y;
}

model {
  for (n in 1:N)
    Y[n] ~ normal(a + b*(X[n]-x0)^2, s_Y);
}

'''

#python interface

data = {
    'N': df.shape[0],
    'X' : df.iloc[:,0],
    'Y' : df.iloc[:,1],
}

#result
model_7_3 = get_stan_model(model_7_3)
params = model_7_3.sampling(data = data)

#데이터 분석을 통해 x0의 범위를 0에서 30으로 정해줌


이차함수가 아닌, $y = a + exp^(-bt)$와 같이 지수분포를 따르는 경우도 있다. 다음은 시간에 따른 혈중 약 농도 자료이다.

In [9]:
#data
df = pd.read_csv('data/data-conc.txt')

#model
model_5_4 ='''
data {
    int T;
    vector [T] Time;
    vector [T] Y;
}

parameters {
    real <lower = 0, upper = 100> a;
    real <lower = 0, upper = 5> b;
    real <lower = 0> sigma;
}

model {
    Y ~ normal(a + b * exp(-b * Time), sigma);
}
        

'''

#python interface

data = {
    'T': df.shape[0],
    'Time' : df.iloc[:,0],
    'Y' : df.iloc[:,1],
}

#result
model_5_4 = get_stan_model(model_5_4)
params = model_5_4.sampling(data = data)


## 7.4 다중공산성

> 다중회귀에서 독립변수 간 상관이 커 회귀계수가 수렴하지 않는 경우

$$\mu [n] =  b_1 + b_2 * A[n] + b_3 * B[n]  $$



A와  B의 상관계수가 0.8 정도 이상이 되면 위 모형은 다음 모형과 유사해진다.

$$\mu [n] =  b_1 + (b_2 + b_3) * A[n]  $$

즉, b_2, b_3가 유일하게 결정되지 않아 한 값으로 수렴하지 않는다. 이럴 경우 A와 B 중 한 독립변수만을 선택한다.

## 7.5 중첩

> 종속변수와 독립변수에 모두 영향을 주는 변수가 모형 밖에 존재할때 (Confounding)

예를 들어 "나이"라는 변수가 "체중"과 "달리기 속도"의 공통원인변수인데, 이를 모형에 포함하지 않을경우 "체중"증가에 따라 "달리기 속도"가 증가한다는 결론을 얻을 수 있다. 본 상황에서는 다음과 같이 모델링이 적절하며, 여러 회귀를 조합해 변수 간 인과관계 모색하는 경로분석을 통해 적절한 모형설정이 가능하다.

$$
\begin{array}{ll}{\mu_{\text {Weight }}[n]=c_{1}+c_{2} A g e[n]} & {n=1, \ldots, N} \\ {\text { Weight }[n] \sim \text { Normal }\left(\mu_{\text {Weight }}[n], \sigma_{W}\right)} & {n=1, \ldots, N} \\ {\mu_{Y}[n]=b_{1}+b_{2} A g e[n]+b_{3} \text { Weight }[n]} & {n=1, \ldots, N} \\ {Y[n] \sim \text { Normal }\left(\mu_{Y}[n], \sigma_{Y}\right)} & {n=1, \ldots, N}\end{array}
$$

이외에도 

7.6 설명변수가 너무 많을때 lasso나 pca등을 이용해 변수를 선택하기

7.7 measurement error를 고려해 설명변수에 노이즈를 포함하기

7.8 censor 데이터의 경우 두 상황을 한 model block에서 target에 반영하기(아래)

<pre><code>
model {
  for (n in 1:N_obs)
    Y_obs[n] ~ normal(mu, s_Y);
  target += N_cens * normal_lcdf(L | mu, s_Y);
}

</pre></code>

7.9 outlier 모형화

outlier는 세 가지 방식으로 처리할 수 있다. 
outlier의 경우 전처리를 통해 outlier를 완전히 제거
outlier발생에 robust한 모형을 만드는 방식
outlier를 outlier로 보지 않고 outlier 발생기작을 모형화

두번째는 정규분포 대신 t분포(or 코시분포- 자유도1인 t분포)를 이용할 수 있다. 

세번째의 경우 혼합정규분포나 영과잉 푸아송분포를 이용할 수 있다.